In [29]:
# 필요한 라이브러리 import

import folium
import numpy as np
import pandas as pd

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import geopandas as gpd

#시각화 모듈
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns


# 한글 글자 깨짐 방지
plt.rcParams["font.family"] = 'Nanum Gothic'
sns.set(font="Malgun Gothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')


# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

In [30]:
data1 = pd.read_csv('1.대전광역시_교통사고내역(2017~2019).csv')
data12 = gpd.read_file('12.대전광역시_인구정보(총인구).geojson')
data13 = gpd.read_file('13.대전광역시_인구정보(고령).geojson')
data14 = gpd.read_file('14.대전광역시_인구정보(생산가능).geojson')
data15 = gpd.read_file('15.대전광역시_인구정보(유소년).geojson')
data16 = pd.read_csv('16.대전광역시_기상데이터(2017~2019).csv')
data18 = gpd.read_file('18.대전광역시_교통노드(2018).geojson')
data20 = pd.read_csv('20.대전광역시_평일_일별_시간대별_추정교통량(2018).csv')
data21 = pd.read_csv('21.대전광역시_평일_일별_혼잡빈도강도(2018).csv')
data22 = pd.read_csv('22.대전광역시_평일_일별_혼잡시간강도(2018).csv')

# Type1 - gid, geometry로 병합할 수 없는 데이터
## data1

In [31]:
from scipy.stats import mode
data1 = pd.read_csv('1.대전광역시_교통사고내역(2017~2019).csv')

In [32]:
#시군구에서 구,동을 분리
data1['구'] = [x[1] for x in data1['시군구'].str.split(' ')]
data1['동'] = [x[2] for x in data1['시군구'].str.split(' ')]

#사고유형을 '-'를 기준으로 분리
data1['세부사고유형'] = [x[1] for x in data1['사고유형'].str.split('-')]
data1['사고유형'] = [x[0] for x in data1['사고유형'].str.split('-')]

data1.head()

,사고일,시군구,사고유형,법규위반,사고내용,사망자수,중상자수,경상자수,부상신고자수,가해운전자 차종,가해운전자 연령대,가해운전자 성별,피해운전자 차종,피해운전자 연령대,피해운전자 성별,gid,구,동,세부사고유형
0,2017-01-01,대전광역시 서구 용문동,차대차,안전운전불이행,중상사고,0,1,1,0,승용,50대,남,승용,60대,남,다바905151,서구,용문동,측면충돌
1,2017-01-01,대전광역시 서구 탄방동,차대차,안전운전불이행,경상사고,0,0,1,0,승용,50대,남,승용,30대,남,다바905166,서구,탄방동,추돌
2,2017-01-01,대전광역시 서구 둔산동,차대사람,안전운전불이행,중상사고,0,1,0,0,승용,30대,남,보행자,10대,여,다바888169,서구,둔산동,횡단중
3,2017-01-01,대전광역시 대덕구 오정동,차대차,안전운전불이행,경상사고,0,0,2,0,승용,50대,남,승합,40대,남,다바917174,대덕구,오정동,추돌
4,2017-01-01,대전광역시 유성구 노은동,차대사람,안전운전불이행,경상사고,0,0,1,0,승용,80대,남,보행자,10대미만,남,다바841191,유성구,노은동,횡단중


In [33]:
mode(data1['피해운전자 연령대'])

ModeResult(mode=array(['50대'], dtype=object), count=array([4766]))

In [34]:
mode(data1['가해운전자 연령대'])

ModeResult(mode=array(['50대'], dtype=object), count=array([5750]))

In [35]:
# 연령대 이상값 -> 최빈값
# 피해 운전자 최빈값 50대
# 가해 운전자 최빈값 30대
data1['피해운전자 연령대']=data1['피해운전자 연령대'].replace('미분류','50대')
data1['가해운전자 연령대']=data1['가해운전자 연령대'].replace('기타불명','30대')

# 피해 차종 null값 삭제 
drop_list = data1[data1['피해운전자 차종'].isnull()]
data1.drop(drop_list.index,axis=0,inplace=True)
data1.columns

Index(['사고일', '시군구', '사고유형', '법규위반', '사고내용', '사망자수', '중상자수', '경상자수', '부상신고자수', '가해운전자 차종', '가해운전자 연령대', '가해운전자 성별', '피해운전자 차종', '피해운전자 연령대', '피해운전자 성별', 'gid', '구', '동', '세부사고유형'], dtype='object')

In [36]:
data1 = data1[['사고일', '구', '동','사고유형','세부사고유형', '법규위반', '사고내용', '사망자수', '중상자수', '경상자수','부상신고자수', '가해운전자 차종', '가해운전자 연령대', '가해운전자 성별', '피해운전자 차종', '피해운전자 연령대', '피해운전자 성별', 'gid']]
data1.reset_index(inplace=True)
data1.to_csv('accident_data.csv', index=False)

In [37]:
data1

,index,사고일,구,동,사고유형,세부사고유형,법규위반,사고내용,사망자수,중상자수,경상자수,부상신고자수,가해운전자 차종,가해운전자 연령대,가해운전자 성별,피해운전자 차종,피해운전자 연령대,피해운전자 성별,gid
0,0,2017-01-01,서구,용문동,차대차,측면충돌,안전운전불이행,중상사고,0,1,1,0,승용,50대,남,승용,60대,남,다바905151
1,1,2017-01-01,서구,탄방동,차대차,추돌,안전운전불이행,경상사고,0,0,1,0,승용,50대,남,승용,30대,남,다바905166
2,2,2017-01-01,서구,둔산동,차대사람,횡단중,안전운전불이행,중상사고,0,1,0,0,승용,30대,남,보행자,10대,여,다바888169
3,3,2017-01-01,대덕구,오정동,차대차,추돌,안전운전불이행,경상사고,0,0,2,0,승용,50대,남,승합,40대,남,다바917174
4,4,2017-01-01,유성구,노은동,차대사람,횡단중,안전운전불이행,경상사고,0,0,1,0,승용,80대,남,보행자,10대미만,남,다바841191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22976,23647,2019-12-31,중구,대흥동,차대사람,횡단중,보행자보호의무위반,경상사고,0,0,1,0,승용,50대,남,보행자,20대,여,다바936139
22977,23648,2019-12-31,중구,부사동,차대사람,기타,안전운전불이행,중상사고,0,1,0,0,승용,40대,남,보행자,70대,여,다바941131
22978,23649,2019-12-31,동구,가양동,차대차,추돌,안전운전불이행,중상사고,0,1,1,0,승용,20대,남,승용,50대,여,다바946171
22979,23650,2019-12-31,대덕구,상서동,차대차,기타,안전운전불이행,경상사고,0,0,3,0,승용,30대,남,승용,40대,여,다바929255


## data16 - 날씨 데이터

In [38]:
data16 = pd.read_csv('16.대전광역시_기상데이터(2017~2019).csv')

In [39]:
# 날씨 데이터 이상치 전처리
#이상치 전처리 - 누락치 0으로 치환
data16['일강수량(mm)'].replace('',0,inplace=True) 
data16['일 최심적설(cm)'].replace('',0,inplace=True)
data16['안개 계속시간(hr)'].replace('',0,inplace=True)
data16['평균 풍속(m/s)'].replace('',0,inplace=True)
data16.dropna(subset=['일강수량(mm)'],axis=0,inplace=True) 

In [40]:
# data1의 사고일과 병합때 사용하기 위해 일시 -> 사고일로 rename
data16.rename(columns = {'일시' : '사고일'}, inplace = True)

In [41]:
#문자열로 되어있는 기상데이터를 모두 형변환

data16['평균기온(°C)'] = data16['평균기온(°C)'].astype(float)
data16['최저기온(°C)'] = data16['최저기온(°C)'].astype(float)
data16['최고기온(°C)'] = data16['최고기온(°C)'].astype(float)
data16['일강수량(mm)'] = data16['일강수량(mm)'].astype(float)
data16['최대 풍속(m/s)'] = data16['최대 풍속(m/s)'].astype(float)
data16['최대 풍속 풍향(16방위)'] = data16['최대 풍속 풍향(16방위)'].astype(float)
data16['평균 풍속(m/s)'] = data16['평균 풍속(m/s)'].astype(float)
data16['평균 상대습도(%)'] = data16['평균 상대습도(%)'].astype(float)
data16['일 최심적설(cm)'] = data16['일 최심적설(cm)'].astype(float)
data16['평균 지면온도(°C)'] = data16['평균 지면온도(°C)'].astype(float)
data16['안개 계속시간(hr)'] = data16['안개 계속시간(hr)'].astype(float)
data16.info()
data16.to_csv('weather_data.csv', index=False )

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 7 to 1094
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   사고일             418 non-null    object 
 1   평균기온(°C)        418 non-null    float64
 2   최저기온(°C)        418 non-null    float64
 3   최고기온(°C)        418 non-null    float64
 4   일강수량(mm)        418 non-null    float64
 5   최대 풍속(m/s)      418 non-null    float64
 6   최대 풍속 풍향(16방위)  418 non-null    float64
 7   평균 풍속(m/s)      418 non-null    float64
 8   평균 상대습도(%)      418 non-null    float64
 9   일 최심적설(cm)      38 non-null     float64
 10  평균 지면온도(°C)     418 non-null    float64
 11  안개 계속시간(hr)     31 non-null     float64
dtypes: float64(11), object(1)
memory usage: 42.5+ KB


# type2 - 사고격자와 gid or geometry 로 병합 가능한 데이터

# 데이터 12 : 인구 데이터

In [42]:
#val에 있는 nan값 전처리
data12['val'] =  data12['val'].fillna(0)
data12.to_file('total_population.geojson', driver= 'GeoJSON')

# 데이터 13 : 고령자 데이터 

In [43]:
#val에 있는 nan값 전처리
data13['val'] =  data13['val'].fillna(0)
data12.to_file('old_population.geojson', driver= 'GeoJSON')

# 데이터 14 : 생산가능 인구 

In [44]:
#val에 있는 nan값 전처리
data14['val'] =  data14['val'].fillna(0)
data14.to_file('labor_population.geojson', driver= 'GeoJSON')

# 데이터 15 : 유소년 인구

In [45]:
#val에 있는 nan값 전처리
data15['val'] =  data15['val'].fillna(0)
data15.to_file('youth_population.geojson', driver= 'GeoJSON')

# 데이터 16 : 교통노드

In [46]:
data20 = pd.read_csv('20.대전광역시_평일_일별_시간대별_추정교통량(2018).csv')

In [47]:
data20['상세도로망_LinkID'] = data20['상세도로망_LinkID'].astype(str)
data20['상세도로망_LinkID'] = data20['상세도로망_LinkID'].str[:-2]
data20['상세도로망_LinkID'] = data20['상세도로망_LinkID'].astype(int)
#data20.to_csv('node_data.csv')

In [48]:
data21 = data20[data20['시간적범위']=='fulltime']

In [49]:
# 전체 시간 (24시간 대비 평균 교통량)
data21 = data21[['상세도로망_LinkID','전체_추정교통량','승용차_추정교통량', '버스_추정교통량', '화물차_추정교통량']].groupby(['상세도로망_LinkID']).mean()
data21.reset_index(inplace=True)

In [50]:
data21.to_csv('node_data.csv',index=False)

# 데이터 17 : 평일 일별 혼잡 빈도 강도

In [51]:
data22 = pd.read_csv('21.대전광역시_평일_일별_혼잡빈도강도(2018).csv')

In [52]:
data22['상세도로망_LinkID'] = data22['상세도로망_LinkID'].astype(str)
data22['상세도로망_LinkID'] = data22['상세도로망_LinkID'].str[:-2]
data22['상세도로망_LinkID'] = data22['상세도로망_LinkID'].astype(int)

data22 = data22[['상세도로망_LinkID','혼잡빈도강도']].groupby(['상세도로망_LinkID']).mean()
data22.reset_index(inplace=True)

In [53]:
data22.to_csv('honjap_frequency.csv', index=False)

#  데이터 18 : 평일 일별 혼잡 시간 강도

In [54]:
data23 = pd.read_csv('22.대전광역시_평일_일별_혼잡시간강도(2018).csv')

In [55]:
data23['상세도로망_LinkID'] = data23['상세도로망_LinkID'].astype(str)
data23['상세도로망_LinkID'] = data23['상세도로망_LinkID'].str[:-2]
data23['상세도로망_LinkID'] = data23['상세도로망_LinkID'].astype(int)

data23 = data23[['상세도로망_LinkID','혼잡시간강도']].groupby(['상세도로망_LinkID']).mean()
data23.reset_index(inplace=True)

In [56]:
data23.to_csv('honjap_intensity.csv',index=False)